In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
#from pyspark.sql.functions import col, unix_timestamp, from_utc_timestamp, current_timestamp


In [0]:
catalogo = "catalog_dev"
esquema_source = "bronze"
esquema_sink = "silver"

In [0]:
# Obtener la fecha actual con la zona horaria de Colombia y formatearla como "yyyyMM"
fecha_actual_archivo = spark.sql("""
    SELECT date_format(
        from_utc_timestamp(current_timestamp(), 'America/Bogota'), 
        'yyyyMMdd'
    ) AS fecha
""").collect()[0]["fecha"]

# Mostrar el resultado
print(fecha_actual_archivo)

20251110


In [0]:
# Lectura de las tablas bronze filtradas para la fecha en que se ejecute el notebook
df_maestro = spark.table(f"{catalogo}.{esquema_source}.maestro_horas") \
    .filter(to_date(from_utc_timestamp("Fecha_Ejecucion", "America/Bogota")) == to_date(from_utc_timestamp(current_timestamp(), "America/Bogota")))

df_bitacora = spark.table(f"{catalogo}.{esquema_source}.bitacora_incidentes") \
    .filter(to_date(from_utc_timestamp("Fecha_Ejecucion", "America/Bogota")) == to_date(from_utc_timestamp(current_timestamp(), "America/Bogota")))


#### Filtros con la tabla bitacora_incidentes <a class="anchor" id="seccion3"></a>

In [0]:
#Filtra el dataframe df_bitacora sólo para los incidentes activos
df_bitacora = df_bitacora.filter((col("Estado")=="Activo"))

In [0]:
# Join entre la sábana de datos Bitácora y el Maestro de tiempos

df_union = df_maestro.join(df_bitacora, ["Zona"], "inner")

df_union = df_union.select(
    "Cod_incidente",
    "Zona",
    "Estado",
    "Hora_Asignacion",
    "Tipo_Vehiculo",
    "Numero_Tecnicos",
    "Equipo_Afectado",
    "Tipo_Falla",
    "Prioridad",
    "Lider_Tecnico",
    "Criticidad",
    "Tiempo_Espera_Horas"
)


In [0]:
#Función que recibe un dataframe y le agrega la columna Horas_Transcurridas
def agregar_horas_transcurridas(df):
    hora_colombiana = from_utc_timestamp(current_timestamp(), "America/Bogota")
    return df.withColumn(
        "Horas_Transcurridas",
        ((unix_timestamp(hora_colombiana) - unix_timestamp(col("Hora_Asignacion"))) / 3600).cast("int")
    )


In [0]:
df_diferencia_horas = agregar_horas_transcurridas(df_union)

In [0]:
# Clasificación para Tiempos de Espera

# Crear la columna Clasificacion en función de Horas_Transcurridas y Tiempo_Espera_Horas
df_Criticidad = df_diferencia_horas.withColumn(
    "Clasificacion", 
    F.when((F.col("Horas_Transcurridas") >= F.col("Tiempo_Espera_Horas")) & (F.col("Criticidad") == "Alta"), "Alta")
     .when((F.col("Horas_Transcurridas") >= F.col("Tiempo_Espera_Horas")) & (F.col("Criticidad") == "Media"), "Media")
     .when((F.col("Horas_Transcurridas") >= F.col("Tiempo_Espera_Horas")) & (F.col("Criticidad") == "Baja"), "Baja")
)

# Crear un índice de prioridad para cada clasificación
df_with_priority = df_Criticidad.withColumn(
    "priority", 
    F.when(F.col("Clasificacion") == "Alta", 3)
     .when(F.col("Clasificacion") == "Media", 2)
     .when(F.col("Clasificacion") == "Baja", 1)
)

# Especificar la ventana para conservar el valor de mayor prioridad para cada Horas_Transcurridas
window_spec = Window.partitionBy("Cod_incidente").orderBy(F.col("priority").desc())

# Filtrar para mantener solo la clasificación con mayor prioridad
df_ranked = df_with_priority.withColumn("rank", F.row_number().over(window_spec))
df_clasificacion_final = df_ranked.filter(F.col("rank") == 1).drop("rank", "priority")
df_clasificacion_final = df_clasificacion_final.withColumn("Horas_Transcurridas", round(df_clasificacion_final["Horas_Transcurridas"], 2))
df_clasificacion_final = df_clasificacion_final.filter(col("Clasificacion").isNotNull())

In [0]:
# Convertir la hora actual (UTC) a hora colombiana
df_clasificacion_final = df_clasificacion_final.withColumn(
    "Fecha_Ejecucion", 
    from_utc_timestamp(current_timestamp(), "America/Bogota")
)

# Formatear la fecha en formato legible
df_clasificacion_final = df_clasificacion_final.withColumn(
    "Fecha_Ejecucion", 
    date_format("Fecha_Ejecucion", "yyyy-MM-dd HH:mm:ss")
)

In [0]:
# Obtener fecha actual en hora Colombia
fecha_colombia = spark.sql("SELECT to_date(from_utc_timestamp(current_timestamp(), 'America/Bogota')) AS fecha").collect()[0]["fecha"]

# Borra los registros de la tabla bitacora_clasificacion que coincidan con la fecha actual
spark.sql(f"""
DELETE FROM catalog_dev.silver.bitacora_clasificacion
WHERE to_date(from_utc_timestamp(Fecha_Ejecucion, 'America/Bogota')) = DATE('{fecha_colombia}')
""")

DataFrame[num_affected_rows: bigint]

In [0]:
# Obtener el esquema de la tabla bitacora_clasificacion
target_schema_bitacora = spark.table(f"{catalogo}.{esquema_sink}.bitacora_clasificacion").schema

# Castear el dataframe a ese esquema
df_bitacora_casted = df_clasificacion_final.select(
    [col(field.name).cast(field.dataType).alias(field.name) for field in target_schema_bitacora]
)

df_bitacora_casted.write.mode("append").saveAsTable(f"{catalogo}.{esquema_sink}.bitacora_clasificacion")